# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [9]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2023-04-23	Decoding human spontaneous spiking activity in medial temporal lobe from scalp EEG	iScience	Linking scalp electroencephalography (EEG) signals and spontaneous firing activity from deep nuclei in humans is not trivial. This study analyzed simultaneous recordings of scalp EEG and unit activity in medial temporal lobe areas (amygdala and hippocampus). Using linear regression models, the research demonstrated that changes in neuronal firing activity could be estimated from EEG frequency modulations, revealing region-specific dynamics during wakefulness and non-REM sleep.	Shpigelman L. (2023). "Decoding human spontaneous spiking activity in medial temporal lobe from scalp EEG." <i>iScience</i>.	decoding-human-spontaneous-spiking	https://www.cell.com/iscience/fulltext/S2589-0042(23)00468-6	
2017-01-01	Epidemiological Models Lacking Process Noise Can Be Overconfident	Informatics for Health	Highlighted the importanc

## Import pandas

We are using the very handy pandas library for dataframes.

In [10]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [11]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2023-04-23,Decoding human spontaneous spiking activity in...,iScience,Linking scalp electroencephalography (EEG) sig...,"Shpigelman L. (2023). ""Decoding human spontane...",decoding-human-spontaneous-spiking,https://www.cell.com/iscience/fulltext/S2589-0...,NaN
1,2017-01-01,Epidemiological Models Lacking Process Noise C...,Informatics for Health,Highlighted the importance of using stochastic...,"Shpigelman L. (2017). ""Epidemiological Models ...",epidemiological-models-noise,https://ebooks.iospress.nl/publication/46317,NaN
2,2016-08-01,Psychophysiological whole-brain network cluste...,Human Brain Mapping,"Introduced a novel method, CRNDA, for delineat...","Shpigelman L. (2016). ""Psychophysiological who...",brain-network-clustering,https://pmc.ncbi.nlm.nih.gov/articles/PMC6867387/,NaN
3,2014-01-01,Estimating the impact of prevention action: a ...,MIE,Proposed a Bayesian simulation model for estim...,"Shpigelman L. (2014). ""Estimating the impact o...",cervical-cancer-simulation,https://www.sciencedirect.com/science/article/...,NaN
4,2012-01-01,Estimating BOLD Signals of Deep Brain Networks...,OHBM,Developed a method to estimate deep brain netw...,"Shpigelman L. (2012). ""Estimating BOLD Signals...",bold-signals-eeg,NaN,NaN
5,2011-01-01,Smarter Log Analysis,IBM Journal of Research and Development,Presented the MELODY solution for analyzing ex...,"Shpigelman L. (2011). ""Smarter Log Analysis."" ...",smarter-log-analysis,https://ieeexplore.ieee.org/abstract/document/...,NaN
6,2008-01-01,Kernel-ARMA for Hand Tracking and Brain-Machin...,Advances in Neural Information Processing Syst...,Adapted and implemented the KARMA algorithm to...,"Shpigelman L. (2008). ""Kernel-ARMA for Hand Tr...",kernel-arma-hand-tracking,https://papers.nips.cc/paper_files/paper/2008/...,NaN
7,2006-01-01,Nearest Neighbor Based Feature Selection for R...,Advances in Neural Information Processing Syst...,Proposed a k-nearest neighbor-based feature se...,"Shpigelman L. (2006). ""Nearest Neighbor Based ...",nearest-neighbor-feature-selection,https://papers.nips.cc/paper_files/paper/2005/...,NaN
8,2005-01-01,Spikernels: Predicting Arm Movements by Embedd...,Neural Computation,"Developed Spikernels, biologically motivated k...","Shpigelman L. (2005). ""Spikernels: Predicting ...",spikernels,https://direct.mit.edu/neco/article-abstract/1...,NaN
9,2004-01-01,A Temporal Kernel-Based Model for Tracking Han...,Advances in Neural Information Processing Syst...,Introduced a temporal kernel-based system for ...,"Shpigelman L. (2004). ""A Temporal Kernel-Based...",temporal-kernel-hand-tracking,https://papers.neurips.cc/paper_files/paper/20...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [12]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [13]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [14]:
!ls ../_publications/

2004-01-01-temporal-kernel-hand-tracking.md
2005-01-01-spikernels.md
2006-01-01-nearest-neighbor-feature-selection.md
2008-01-01-kernel-arma-hand-tracking.md
2011-01-01-smarter-log-analysis.md
2012-01-01-bold-signals-eeg.md
2014-01-01-cervical-cancer-simulation.md
2016-08-01-brain-network-clustering.md
2017-01-01-epidemiological-models-noise.md
2023-04-23-decoding-human-spontaneous-spiking.md


In [15]:
!cat ../_publications/2004-01-01-temporal-kernel-hand-tracking.md

---
title: "A Temporal Kernel-Based Model for Tracking Hand Movements from Neural Activities"
collection: publications
permalink: /publication/2004-01-01-temporal-kernel-hand-tracking
excerpt: 'Introduced a temporal kernel-based system for dynamic hand-tracking from neural activity. Showcased superior accuracy over static methods, enabling effective BMI-based motor control in real-time experiments.'
date: 2004-01-01
venue: 'Advances in Neural Information Processing Systems (NIPS)'
paperurl: 'https://papers.neurips.cc/paper_files/paper/2004/hash/f35a2bc72dfdc2aae569a0c7370bd7f5-Abstract.html'
citation: 'Shpigelman L. (2004). &quot;A Temporal Kernel-Based Model for Tracking Hand Movements from Neural Activities.&quot; <i>Advances in Neural Information Processing Systems</i>.'
---
Introduced a temporal kernel-based system for dynamic hand-tracking from neural activity. Showcased superior accuracy over static methods, enabling effective BMI-based motor control in real-time experiments.

[D